In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
import torch.optim as optim
from torch import nn


# Choose User Path

In [ ]:
from enum import Enum
class Pathes(Enum):
  HADAR = 1
  RAN = 2
  ITAMAR = 3

In [ ]:
import sys

##Just choose here the user
path = Pathes.ITAMAR

if(path == Pathes.HADAR):
  PATH='/content/gdrive/My Drive/IntroToDeep/Project/codes/Model_TextRNN/Weights.pt'
  sys.path.insert(0,'/content/gdrive/My Drive/IntroToDeep/Project/codes/Model_TextRNN')
  train_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/train.txt'
  test_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/test.txt'
  w2v_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/glove.6B.100d.txt'
elif(path == Pathes.RAN):
  PATH='/content/gdrive/My Drive/IntroToDeep/Project/codes/Model_TextRNN/Weights.pt'
  sys.path.insert(0,'/content/gdrive/My Drive/IntroToDeep/Project/codes/Model_TextRNN')
  train_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/train.txt'
  test_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/test.txt'
  w2v_file = '/content/gdrive/My Drive/IntroToDeep/Project/codes/data/glove.6B.100d.txt'
elif(path == Pathes.ITAMAR):
  PATH='/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/Model_TextRNN/Weights.pt'
  sys.path.insert(0,'/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/Model_TextRNN')
  #train_file = '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/data/train.txt'
  train_file = '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/data/new_train.txt'
  test_file = '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/data/test.txt'
  val_file = '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/data/validation.txt'
  w2v_file = '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/data/glove.6B.100d.txt'

# Import code

In [ ]:
from utils import *
from model import *
from config import Config

config = Config()


In [ ]:
#config.max_epochs = 1 #Max 10

# Load Data + Embedding

In [ ]:
##########
#Embedding
##########

dataset = Dataset(config)
dataset.load_data(w2v_file, train_file, test_file, val_file)

100%|█████████▉| 399999/400000 [00:20<00:00, 19316.40it/s]


Loaded 128000 training examples
Loaded 40000 test examples
Loaded 32000 validation examples


# Model

In [ ]:
# Create Model with specified optimizer and loss function
##############################################################
model = TextRNN(config, len(dataset.vocab), dataset.word_embeddings)
if torch.cuda.is_available():
    model.cuda()
model.train()
optimizer = optim.SGD(model.parameters(), lr=config.lr)
NLLLoss = nn.NLLLoss()
model.add_optimizer(optimizer)
model.add_loss_op(NLLLoss)
##############################################################

# Train

In [ ]:
train_losses = []
val_accuracies = []

for i in range(config.max_epochs):
    print ("Epoch: {}".format(i))
   
    train_loss,val_accuracy = model.run_epoch(dataset.train_iterator, dataset.val_iterator, i)
    train_losses.append(train_loss)
    val_accuracies.append(val_accuracy)

train_acc = evaluate_model(model, dataset.train_iterator)
val_acc = evaluate_model(model, dataset.val_iterator)
test_acc = evaluate_model(model, dataset.test_iterator)

print ('Final Training Accuracy: {:.4f}'.format(train_acc))
print ('Final Validation Accuracy: {:.4f}'.format(val_acc))
print ('Final Test Accuracy: {:.4f}'.format(test_acc))


#save weights
print('--------Weights saved------------')
torch.save(model, PATH)

#Metrices


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
def evaluate_model_metrices(model, iterator):
    y_all_preds = []
    y_all = []
    for idx,batch in enumerate(iterator):
        if torch.cuda.is_available():
            x = batch.text.cuda()
        else:
            x = batch.text
        #raw value from the model
        y_pred = model(x)
        #Round the raw value to 1 or 2 - cutoff Hyperparameter?
        predicted = torch.max(y_pred.cpu().data, 1)[1] + 1

        #extend all Y_lables, Y_predicted to vectors
        y_all_preds.extend(predicted.numpy())
        y_all.extend(batch.label.numpy())

    accuracy = accuracy_score(y_all, np.array(y_all_preds).flatten())
    #precision, recall, fscore = precision_recall_fscore_support(y_all, y_all_preds, average='binary')
    #returns: score - Accuracy score, 
    #         predicted - Array of predicted raw values (just the last batch)
    #         y_all - all the y labels in a vector
    #         y_all_preds - all the predicted values rounded to 1,2
    #         precision, recall, fscore
    return accuracy, predicted, y_all, y_all_preds, #precision, #recall, fscore

# New Section

In [ ]:
#get vectors
accuracy, predicted, Y_all, Y_all_Preds = evaluate_model_metrices(model, dataset.test_iterator)

In [ ]:
#Precision/Recall
precision_recall_fscore_support(Y_all, Y_all_Preds, average='binary')

In [ ]:
#Classification Report
print(classification_report(Y_all, Y_all_Preds))

In [ ]:
#Confusion Matrix
cm = confusion_matrix(Y_all, Y_all_Preds, labels=[1,2])
print(cm)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))

heatmap = sns.heatmap(cm, annot=True, fmt="d")
heatmap.set_title('confusion matrix TextRNN', fontdict={'fontsize':18}, pad=12);

# Graphs


In [ ]:
#append to one list
train_loss1 = []
val_accuracies_appended = []

#run until max_epochs 
for i in range(0,config.max_epochs ):
  train_loss1.extend(train_losses[i])
  val_accuracies_appended.extend(val_accuracies[i])

print(train_loss1) 
print(len(train_loss1))
print(val_accuracies_appended) 
print(len(val_accuracies_appended))

In [ ]:
train_loss2 = []
train_loss2 = [x * (-1) for x in train_loss1]

In [ ]:
#print graphs
from matplotlib import pyplot
pyplot.title('Train and Validation Accuracy TextRNN')
pyplot.plot(train_loss2, label='train')
pyplot.plot(val_accuracies_appended, label='validation')
pyplot.legend()
pyplot.show()

# Load model to verify

In [ ]:
#load model 
model = torch.load(PATH)
model.eval()

TextRNN(
  (embeddings): Embedding(50440, 100)
  (lstm): LSTM(100, 32, num_layers=2, dropout=0.8, bidirectional=True)
  (dropout): Dropout(p=0.8, inplace=False)
  (fc): Linear(in_features=128, out_features=2, bias=True)
  (softmax): Softmax(dim=-1)
  (loss_op): NLLLoss()
)

In [ ]:
#Get Data for Last layer
#Train Set
accuracy_train, predicted_train, Y_all_train, Y_all_Preds_train = evaluate_model_metrices(model, dataset.train_iterator)
#Test Set
accuracy_test, predicted_test, Y_all_test, Y_all_Preds_test = evaluate_model_metrices(model, dataset.test_iterator)

In [ ]:
#Classification Report
print(classification_report(Y_all_test, Y_all_Preds_test))

              precision    recall  f1-score   support

           1       0.92      0.88      0.90     20082
           2       0.89      0.92      0.90     19918

    accuracy                           0.90     40000
   macro avg       0.90      0.90      0.90     40000
weighted avg       0.90      0.90      0.90     40000



In [ ]:
#Classification Report
print(classification_report(Y_all_train, Y_all_Preds_train))

              precision    recall  f1-score   support

           1       0.92      0.89      0.90     63819
           2       0.89      0.92      0.91     64181

    accuracy                           0.90    128000
   macro avg       0.91      0.90      0.90    128000
weighted avg       0.91      0.90      0.90    128000



In [ ]:
import numpy as np
from numpy import asarray

In [ ]:
Y_RNN_Pred_Test = asarray(Y_all_Preds_test, dtype = 'int32')
Y_RNN_GroundProof_Test = asarray(Y_all_test, dtype = 'int32')
Y_RNN_Pred_Train = asarray(Y_all_Preds_train, dtype = 'int32')
Y_RNN_GroundProof_Train = asarray(Y_all_train, dtype = 'int32')

In [ ]:
#Save the tensors
%cd '/content/gdrive/My Drive/Colab Notebooks/Uni/IntroToDeep/IntroToDeep/Project/codes/Model_TextRNN/'
np.save('Y_RNN_Pred_Test', Y_RNN_Pred_Test)
np.save('Y_RNN_GroundProof_Test', Y_RNN_GroundProof_Test)
np.save('Y_RNN_Pred_Train', Y_RNN_Pred_Train)
np.save('Y_RNN_GroundProof_Train', Y_RNN_GroundProof_Train)

/content/gdrive/.shortcut-targets-by-id/1XNTZrFPo5LZhTNZ3l5LoLnleAlL4h-1v/IntroToDeep/Project/codes/Model_TextRNN


In [ ]:
#Classification Report
print(classification_report(Y_RNN_GroundProof_Test, Y_RNN_Pred_Test))